In [39]:
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.externals import joblib
from imutils import paths
import numpy as np
import imutils
import cv2
import os

In [40]:
# resize the image to a fixed size, then flatten the image into
# a list of raw pixel intensities
def image_to_feature_vector(image, size = (100, 50)):
    return cv2.resize(image, size).flatten()

def extract_color_histogram(image, bins = (8, 8, 8)):
    # extract a 3D color histogram from the HSV color space using
    # the supplied number of `bins` per channel
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    hist = cv2.calcHist([hsv], [0, 1, 2], None, bins, [0, 180, 0, 256, 0, 256])

    # handle normalizing the histogram if we are using OpenCV 2.4.X
    if imutils.is_cv2():
        hist = cv2.normalize(hist)

    # otherwise, perform "in place" normalization in OpenCV 3 (I
    # personally hate the way this is done
    else:
        cv2.normalize(hist, hist)

    # return the flattened histogram as the feature vector
    return hist.flatten()

dataset = 'cheekForKNN'
# initialize the raw pixel intensities matrix, the features matrix,
# and labels list
rawImages = []
features = []
labels = []

# loop over the input images
for dirname, dirnames, filenames in os.walk(dataset):
    if dirname != dataset:
        path, label = os.path.split(dirname)
        # grab the list of images that we'll be describing
        imagePaths = list(paths.list_images(dirname))
        for (i, imagePath) in enumerate(imagePaths):
        # load the image and extract the class label (assuming that our
        # path as the format: /dataset/{class}/{image_num}.jpg
            image = cv2.imread(imagePath)

            # extract raw pixel intensity "features", followed by a color
            # histogram to characterize the color distribution of the pixels
            # in the image
            pixels = image_to_feature_vector(image)
            hist = extract_color_histogram(image)

            # update the raw images, features, and labels matricies,
            # respectively
            rawImages.append(pixels)
            features.append(hist)
            labels.append(label)

            # show an update every 1,000 images
            if i > 0 and i % 1000 == 0:
                print("[INFO] processed {}/{}".format(i, len(imagePaths)))

In [49]:
#train_rX, train_rY = rawImages, labels
(train_rX, test_rX, train_rY, test_rY) = train_test_split(rawImages, labels, test_size = 0.2)
(train_fX, test_fX, train_fY, test_fY) = train_test_split(features, labels, test_size = 0.2)

In [60]:
#kernel =‘linear’, ‘poly’, ‘rbf’(default)
model_r = SVC(kernel = 'poly', probability = True, random_state = 77)
model_r.fit(train_rX, train_rY)
test_rX_pred = model_r.predict(test_rX)
print('Accuracy_r: %.2f' % accuracy_score(test_rY, test_rX_pred))

model_f = SVC(kernel = 'linear', probability = True, random_state = 77)
model_f.fit(train_fX, train_fY)
test_fX_pred = model_f.predict(test_fX)
print('Accuracy_f: %.2f' % accuracy_score(test_fY, test_fX_pred))

Accuracy_r: 0.69
Accuracy_f: 0.66


In [88]:
image = cv2.imread('acne/S__45334552.jpg')
pixels = image_to_feature_vector(image)
print(model_r.predict(pixels.reshape(1, -1)))
print(model_r.predict_proba(pixels.reshape(1, -1)))
print()
hist = extract_color_histogram(image)
print(model_f.predict(hist.reshape(1, -1)))
print(model_f.predict_proba(hist.reshape(1, -1)))

['notgood']
[[ 0.51082129  0.48917871]]

['good']
[[ 0.518847  0.481153]]


In [72]:
joblib.dump(model_r, '20180403_model_svm_rgb_cheek.pkl')
#joblib.dump(model_f, '20180403_model_svm_hsv_cheek.pkl')

['20180403_model_svm_rgb_cheek.pkl']

In [ ]:
raw = joblib.load('20180403_model_svm_rgb_cheek.pkl')
hsv = joblib.load('20180403_model_svm_hsv_cheek.pkl')

image = cv2.imread('acne/S__43925507.jpg')
pixels = image_to_feature_vector(image)
print(raw.predict(pixels.reshape(1, -1)))
print(raw.predict_proba(pixels.reshape(1, -1)))

hist = extract_color_histogram(image)
print(hsv.predict(hist.reshape(1, -1)))
print(hsv.predict_proba(hist.reshape(1, -1)))